In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from tqdm import tqdm

import pickle

import os
import pickle
import neurokit2 as nk
from tqdm import tqdm

from utils.utils import ppg_findpeaks_bishop, rr_interval, exctract_hrv, exctract_hr, extract_p_wave, extract_sgr, ppg_amplitude, ppg_frequency 
plt.rcParams["figure.figsize"] = (20,10)

In [4]:
df_test = pd.read_pickle('new_dfs/df_separated.pkl')

In [5]:
df_test

,Name,block,group,ECG,SGR,PPG,PPG_differential,length
0,12_AlsGA_baseline.txt,baseline,Wrong_feedback,"[-0.0785, -0.069, -0.05225, -0.04825, -0.0565,...","[0.605774, 0.610352, 0.614929, 0.621033, 0.625...","[2.6225, 2.55125, 2.47812, 2.40437, 2.33, 2.25...",NaN,30235
1,12_AlsGA_block1.txt,block1_level1_batch0,Wrong_feedback,"[-0.05625, -0.064, -0.049, -0.041, -0.0535, -0...","[1.46942, 1.48773, 1.50604, 1.52435, 1.54114, ...","[1.805, 1.76188, 1.71875, 1.67563, 1.63188, 1....",NaN,1250
2,12_AlsGA_block1.txt,block1_level1_batch1,Wrong_feedback,"[-0.05225, -0.049, -0.0445, -0.04325, -0.0455,...","[4.1626, 4.19159, 4.21906, 4.24652, 4.27399, 4...","[1.91625, 1.86313, 1.81, 1.75688, 1.70375, 1.6...",NaN,1250
3,12_AlsGA_block1.txt,block1_level1_batch2,Wrong_feedback,"[-0.06425, -0.067, -0.0595, -0.048, -0.04025, ...","[-0.0778198, -0.0808716, -0.0839233, -0.085449...","[0.89125, 0.876875, 0.86375, 0.853125, 0.84562...",NaN,1250
4,12_AlsGA_block1.txt,block1_level1_batch3,Wrong_feedback,"[-0.02475, -0.0375, -0.03275, -0.02725, -0.037...","[-0.552368, -0.553894, -0.556946, -0.559998, -...","[0.666875, 0.661875, 0.6575, 0.653125, 0.64937...",NaN,1250
...,...,...,...,...,...,...,...,...
23915,59_ZolAI_training.txt,training_batch14,Wrong_feedback,"[-0.08975, -0.106, -0.11325, -0.09675, -0.074,...","[0.675964, 0.665283, 0.656128, 0.646973, 0.636...","[-0.508125, -0.5125, -0.5175, -0.52125, -0.524...",NaN,1250
23916,59_ZolAI_training.txt,training_batch15,Wrong_feedback,"[-0.0275, -0.00925, -0.0165, -0.02125, -0.0157...","[-0.790405, -0.782776, -0.775146, -0.765991, -...","[0.39375, 0.388125, 0.38125, 0.374375, 0.3675,...",NaN,1250
23917,59_ZolAI_training.txt,training_batch16,Wrong_feedback,"[0.0345, 0.04575, 0.08625, 0.08125, 0.055, 0.0...","[1.7746, 1.78528, 1.79443, 1.80511, 1.8158, 1....","[-0.3675, -0.37625, -0.384375, -0.39125, -0.39...",NaN,1250
23918,59_ZolAI_training.txt,training_batch17,Wrong_feedback,"[-0.01725, -0.035, -0.07475, -0.08825, -0.0695...","[0.67749, 0.665283, 0.654602, 0.642395, 0.6317...","[0.138125, 0.1675, 0.19375, 0.2175, 0.23875, 0...",NaN,1250


In [6]:
df_test.drop('PPG_differential',axis =1, inplace = True)

In [7]:
names = []
for name in df_test['Name']:
    names.append(name[3:8])
    
unique_names = np.unique(names, axis=0)
unique_names

array(['AlsGA', 'AmrVV', 'AntAA', 'AntEG', 'AntMI', 'AsiDI', 'AziAV',
       'BabYV', 'BarEA', 'BarSR', 'BurAV', 'ButAS', 'ChiAA', 'DavAS',
       'DoiNA', 'EgoAP', 'ElcRD', 'FedYA', 'GaiMR', 'GalSK', 'GanTD',
       'GeiEA', 'GibGR', 'GlaNA', 'GorEA', 'GulAS', 'IboSN', 'IotMY',
       'IvaVV', 'KacPM', 'KalEY', 'KatME', 'KirDA', 'KorMN', 'KosPV',
       'KozVA', 'KruIA', 'KudDY', 'KulDS', 'KulSS', 'LeoAD', 'LodAV',
       'MarOV', 'MarYA', 'MasAV', 'MazAA', 'MilSG', 'MitDG', 'MitVG',
       'NekAK', 'NemMS', 'NikAO', 'PavMA', 'PavSO', 'PorGV', 'ProNI',
       'RosSM', 'SarMA', 'ShiAA', 'SidFA', 'SmiYO', 'SneOP', 'SteDI',
       'SteKO', 'StePI', 'SteVR', 'StoIA', 'SubAA', 'SyaEG', 'TihYA',
       'TimSD', 'TimYE', 'TolDI', 'TroAA', 'TseSA', 'TumNY', 'UskDA',
       'UskNA', 'VolAR', 'ZelAS', 'ZolAI'], dtype='<U5')

In [9]:
for name in unique_names:
    tmp = df_test[df_test['Name'].str.contains(name)]
    print(name, len(tmp[tmp.block == 'baseline']))

AlsGA 0
AmrVV 0
AntAA 0
AntEG 0
AntMI 0
AsiDI 0
AziAV 0
BabYV 0
BarEA 0
BarSR 0
BurAV 0
ButAS 0
ChiAA 0
DavAS 0
DoiNA 0
EgoAP 0
ElcRD 0
FedYA 0
GaiMR 0
GalSK 0
GanTD 0
GeiEA 0
GibGR 0
GlaNA 0
GorEA 0
GulAS 0
IboSN 0
IotMY 0
IvaVV 0
KacPM 0
KalEY 0
KatME 0
KirDA 0
KorMN 0
KosPV 0
KozVA 0
KruIA 0
KudDY 0
KulDS 0
KulSS 0
LeoAD 0
LodAV 0
MarOV 0
MarYA 0
MasAV 0
MazAA 0
MilSG 0
MitDG 0
MitVG 0
NekAK 0
NemMS 0
NikAO 0
PavMA 0
PavSO 0
PorGV 0
ProNI 0
RosSM 0
SarMA 0
ShiAA 0
SidFA 0
SmiYO 0
SneOP 0
SteDI 0
SteKO 0
StePI 0
SteVR 0
StoIA 0
SubAA 0
SyaEG 0
TihYA 0
TimSD 0
TimYE 0
TolDI 0
TroAA 0
TseSA 0
TumNY 0
UskDA 0
UskNA 0
VolAR 0
ZelAS 0
ZolAI 0


In [5]:
def divide_chunks(l, n):
    
    """Auxiliary function for dividing batches"""
    
    # looping till length l
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [6]:
def separate_for_seconds(df, ticks):
    new_df = pd.DataFrame(columns = df.columns)
    
    for i in range(len(df)):
        if df['block'][i] == 'baseline':
            row = {'Name' : df['Name'][i],
                   'block' : df['block'][i],
                   'group' : df['group'][i],
                   'ECG' : df['ECG'][i],
                   'SGR' : df['SGR'][i],
                   'PPG' : df['PPG'][i],
                   'length' : df['length'][i]}
            new_df = pd.concat([new_df,pd.DataFrame([row])],ignore_index = True)
        else:
            ECG_batches = list(divide_chunks(df['ECG'][i],ticks))
            SGR_batches = list(divide_chunks(df['SGR'][i],ticks))
            PPG_batches = list(divide_chunks(df['PPG'][i],ticks))
            
            ECG_batches = [x for x in ECG_batches if len(x) >= 250]
            SGR_batches = [x for x in SGR_batches if len(x) >= 250]
            PPG_batches = [x for x in PPG_batches if len(x) >= 250]
             
            ECG_batches = [sublist[:-(len(sublist) % 250)] if len(sublist) % 250 != 0 else sublist for sublist in ECG_batches]
            SGR_batches = [sublist[:-(len(sublist) % 250)] if len(sublist) % 250 != 0 else sublist for sublist in SGR_batches]
            PPG_batches = [sublist[:-(len(sublist) % 250)] if len(sublist) % 250 != 0 else sublist for sublist in PPG_batches]
            
            k = 0 #batch number
            
            for ecg_batch, sgr_batch, ppg_batch in zip(ECG_batches, SGR_batches, PPG_batches):
                row = {'Name' : df['Name'][i],
                   'block' : '{}_batch{}'.format(df['block'][i],k),
                   'group' : df['group'][i],
                   'ECG' : ecg_batch,
                   'SGR' : sgr_batch,
                   'PPG' : ppg_batch,
                   'length' : len(ecg_batch)}
                new_df = pd.concat([new_df,pd.DataFrame([row])],ignore_index = True)
                #new_df = new_df.append(row,ignore_index = True)
                k+=1
    return new_df
            
            
            

In [7]:
def separate(df,unique_names):
    output = pd.DataFrame()
    for name in tqdm(unique_names):
        tmp = df[df['Name'].str.contains(name)]
        tmp = tmp.reset_index(drop = True)
        tmp_out = separate_for_seconds(tmp,1250)
        output = pd.concat([output,tmp_out], ignore_index = True)
        #output = output.append(tmp_out, ignore_index = True)
    return output 

In [8]:
df_new = separate(df_test,unique_names)

100%|███████████████████████████████████████████| 81/81 [00:11<00:00,  7.36it/s]


In [9]:
df_new.to_pickle('new_df/df_separated_batch.pkl')